# Código para detectar codificación del txt

In [ ]:
import chardet  # chardet helps to detect the character encoding

def read_file_with_unknown_encoding(filename):
    try:
        # Use chardet to detect the encoding
        with open(filename, 'rb') as f:
            rawdata = f.read(64)
            result = chardet.detect(rawdata)
            encoding = result['encoding']

        # Now, read the file with the detected encoding
        with open(filename, 'r', encoding=encoding) as file:
            content = file.read(64)
            print(f"Successfully read with encoding: {encoding}")
            return content

    except UnicodeDecodeError:
        print(f"Failed with encoding: {encoding}")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    return None  # appropriate error handling based on your application's logic.



prueba con utf8

In [ ]:
# Use the function
file_content = read_file_with_unknown_encoding('/content/result.txt')
#if file_content is not None:
#    print(file_content)

Successfully read with encoding: UTF-16


prueba con utf16

In [ ]:

# Use the function
file_content = read_file_with_unknown_encoding('/content/drive/MyDrive/codif/file_utf16.txt')
if file_content is not None:
    print(file_content)

Successfully read with encoding: UTF-16
EnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarcIbañezEnekoTelecoMarc

prueba con utf32

In [ ]:
# Use the function
file_content = read_file_with_unknown_encoding('/content/file_utf32.txt')
if file_content is not None:
    print(file_content)

Successfully read with encoding: UTF-32
Hello, this is a test file with UTF-32 encoding.


# HERE WE TRY ZLIB AND LZ4

In [1]:
pip install lz4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.1 MB/s eta 0:00:00


In [2]:
!pip install ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.2 MB/s eta 0:00:00


In [3]:
%load_ext autotime

time: 2.43 ms (started: 2023-12-13 00:12:49 +00:00)


In [4]:
import zlib

time: 491 µs (started: 2023-12-13 00:12:51 +00:00)


In [5]:
import lz4.frame

time: 4.79 ms (started: 2023-12-13 00:12:52 +00:00)


In [7]:
def openFile(path):
    try:
        #file_path = glob.glob(path + '*.txt')[0]
        file_path = "/content/MTP-F21-SRI-A-TX.txt"
        with open(file_path, "rb") as file:
            strF = file.read()
        return strF
    except IndexError:
        print("No .txt file found.")
        return None
    except IOError as e:
        print(f"Error opening file: {e}")
        return None
def fragmentFile(string, length):
    return list(string[0+i: length+i] for i in range(0, len(string), length))
def writeFile(path, buff):
    file = open(path+"result.txt","wb")
    file.write(buff)
    file.close()

time: 969 µs (started: 2023-12-13 00:13:03 +00:00)


In [8]:
strF= openFile("/content/MTP-F21-SRI-A-TX.txt")

time: 2.54 ms (started: 2023-12-13 00:13:11 +00:00)


In [9]:
strF= openFile("/content/MTP-F21-SRI-A-TX.txt")

time: 2.49 ms (started: 2023-12-13 00:13:13 +00:00)


We observe zlib is slower than lz4

In [10]:
compressed_file_zlib = zlib.compress(strF)

time: 19.1 ms (started: 2023-12-13 00:13:16 +00:00)


In [11]:
compressed_file_lz = lz4.frame.compress(strF)

time: 1.23 ms (started: 2023-12-13 00:13:18 +00:00)


But zlib can compress better

In [14]:
zlib_c = 100*len(compressed_file_zlib)/len(strF)
print("File from Pre-Test using zlib was compressed to %.2f " % zlib_c + "%")
lz_c = 100*len(compressed_file_lz)/len(strF)
print("File from Pre-Test using lz4 was compressed to %.2f " % lz_c + "%")

File from Pre-Test using zlib was compressed to 3.35 %
File from Pre-Test using lz4 was compressed to 6.22 %
time: 1.77 ms (started: 2023-12-13 00:18:40 +00:00)


Now we fragment the compressed data and this is what we will send

In [ ]:
payload_size = 32
compressed_fragmented_lz = fragmentFile(compressed_file_lz, payload_size)
#print(type(compressed_fragmented_zlib))

In [ ]:
len(compressed_file_zlib)

363380

now decompress

In [ ]:
msg = zlib.decompress(compressed_file_zlib)
nuevo = compressed_file_zlib[0:330]
msg = zlib.decompress(nuevo)

error: ignored

In [ ]:
compressed_file_lz

In [ ]:
msg_lz = lz4.frame.decompress(compressed_file_lz[0:309])

RuntimeError: ignored

and write file

In [ ]:
writeFile("/content/",msg)

now we check both codecs (Tx and Rx):

In [ ]:
file_content = read_file_with_unknown_encoding('/content/MTP-F21-SRI-A-TX.txt')

In [ ]:
file_content = read_file_with_unknown_encoding('/content/MTP-F23-SRI-A-RX.txt')

In [ ]:
!pwd

/content
time: 107 ms (started: 2023-12-02 00:14:15 +00:00)


comparamos ficheros

In [ ]:
!diff -y --suppress-common-lines /content/MTP-F21-SRI-A-TX-special.txt /content/MTP-F23-SRI-A-RX.txt

In [ ]:
!diff -y --suppress-common-lines /content/el_quijote.txt /content/MTP-F23-SRI-A-RX.txt

# PARTIAL DECOMPRESSION

In [ ]:
!diff -y --suppress-common-lines /content/MTP-F23-MRM-A-RX /content/MTP-F23-MRM-A-TX.txt